In [60]:
import pandas as pd
import eeweather as ee
from folium import plugins
import folium
from folium.plugins import HeatMap

In [2]:
df1 = pd.read_csv('Dataset_1.csv')
df2 = pd.read_csv('Dataset_2.csv')
df3 = pd.read_csv('Dataset_3.csv')

### Assignment #3

In [3]:
df3.shape

(2386, 2)

In [4]:
df3.columns = map(str.lower, df3.columns)

In [5]:
len(set(df3.response_zip_code.to_list()))

2316

In [6]:
df3['zip_5'] = df3['response_zip_code'].astype(str).str[:5].astype(int)
df3.head()

,responseid,response_zip_code,zip_5
0,1000,93440,93440
1,1001,928053811,92805
2,1002,900621538,90062
3,1003,900021746,90002
4,1004,932574653,93257


In [7]:
len(set(df3.zip_5.to_list()))

327

In [8]:
df_zip_summary = df3.groupby('zip_5', as_index=False).count()[['zip_5', 'response_zip_code']]
df_zip_summary.head()

,zip_5,response_zip_code
0,90001,19
1,90002,29
2,90003,27
3,90004,8
4,90006,3


In [23]:
df_zip_summary['LAT'] = ''
df_zip_summary['LNG'] = ''
index_except = []

for index, row in df_zip_summary.iterrows():
    try:
        df_zip_summary.at[index, 'LAT'] = ee.zcta_to_lat_long(row['zip_5'])[0]
        df_zip_summary.at[index, 'LNG'] = ee.zcta_to_lat_long(row['zip_5'])[1]
    except:
        index_except.append(index)
index_except

[]

In [33]:
df_zip_summary.response_zip_code.sum()

2386

In [38]:
df_zip_summary['freq'] = 100*(df_zip_summary['response_zip_code'] / df_zip_summary.response_zip_code.sum())
df_zip_summary.head()

,zip_5,response_zip_code,LAT,LNG,freq
0,90001,19,33.974,-118.25,0.796312
1,90002,29,33.949,-118.247,1.215423
2,90003,27,33.9641,-118.274,1.131601
3,90004,8,34.0762,-118.311,0.335289
4,90006,3,34.048,-118.294,0.125733


In [41]:
df_zip_summary.describe()

,zip_5,response_zip_code,freq
count,327.000000,327.000000,327.000000
mean,91761.036697,7.296636,0.305810
std,1159.129777,8.309619,0.348266
min,90001.000000,1.000000,0.041911
25%,90708.000000,1.000000,0.041911
50%,91761.000000,4.000000,0.167645
75%,92790.500000,9.000000,0.377200
max,93654.000000,37.000000,1.550712


In [63]:
m = folium.Map([34.052235, -118.243683], zoom_start=10)

for index, row in df_zip_summary.iterrows():
    folium.CircleMarker([row['LAT'], row['LNG']],
                        radius=0.25,
                        fill_color="#3db7e4", alpha=0.1
                       ).add_to(m)
    
stationArr = df_zip_summary[['LAT', 'LNG', 'response_zip_code']].values

m.add_child(plugins.HeatMap(stationArr, radius=14))
m

In [59]:
import plotly.express as px
fig = px.density_mapbox(df_zip_summary, 
                        lat='LAT', 
                        lon='LNG', 
                        z='response_zip_code', 
                        radius=20,
                        center=dict(lat=34.052235, lon=-118.243683), 
                        zoom=6,
                        mapbox_style="stamen-terrain")
fig.show()

### Assignment #3

In [ ]:
df2.fillna(0, inplace=True)

In [ ]:
df2.columns = map(str.lower, df2.columns)
df2.columns.to_list()

In [ ]:
df2['phone1len'] = df2['phonenumber1'].astype(str).apply(len)
df2['phonenumber2'] = df2['phonenumber2'].astype(int)
df2['phone2len'] = df2['phonenumber2'].astype(str).apply(len)

In [ ]:
df2.info()

In [ ]:
df2['customerphone'] = ''
index_row = []

for index, row in df2.iterrows():
    try:
        if row['phone1len'] == 10:
            df2.at[index, 'customerphone'] = row['phonenumber1']
        elif (row['phone1len'] < 10) & (row['phone1len']) == 10:
            df2.at[index, 'customerphone'] = row['phonenumber2']
        else:
            df2.at[index, 'customerphone'] = 'invalid_phone'
    except:
        index_row.append(index)
index_row

In [ ]:
df2

### Assignment #1

In [ ]:
df1.info()

In [ ]:
df1.columns = map(str.lower, df1.columns)

In [ ]:
df1['dateyrmo'] = pd.to_datetime(df1['date_of_survey']).dt.to_period('m')

In [ ]:
df1_filter = df1[['dateyrmo', 'surveyquestion1']]
df1_filter.head()

In [ ]:
df1.head()

In [ ]:
ques_1 = ''
ques_2 = ''

for index, row in df1.iterrows():
    if (row['surveyquestion1'] == 4):
        df1.at[index, 'ques_1'] = 1
    if (row['surveyquestion2'] == 4):
        df1.at[index, 'ques_2'] = 1

In [ ]:
df1.head()

In [ ]:
df1.fillna(0, inplace=True)

In [ ]:
df1.head()

In [ ]:
df1['ques_1'] = df1['ques_1'].astype(int)
df1['ques_2'] = df1['ques_2'].astype(int)
df1.head()

In [ ]:
df1_filter = df1[(df1['ques_1'] == 0) & (df1['ques_2'] == 0)].reset_index(False)[['dateyrmo', 'surveyquestion1', 'surveyquestion2']]

In [ ]:
df1_filter.shape

In [ ]:
set(df1_filter.surveyquestion1.to_list())

In [ ]:
set(df1_filter.surveyquestion2.to_list())

In [ ]:
df1_filter.head()

In [ ]:
d1 = pd.crosstab(index=df1_filter['dateyrmo'], columns=[df1_filter['surveyquestion1']]).apply(lambda r: 100*(r/r.sum()), axis=1)

In [ ]:
d2 = pd.crosstab(index=df1_filter['dateyrmo'], columns=[df1_filter['surveyquestion2']]).apply(lambda r: 100*(r/r.sum()), axis=1)

In [ ]:
df = pd.concat({d.columns.name: d for d in [d1, d2]}, axis=1)
netScore1 = df.round(decimals=3)['surveyquestion1'][5] - (df.round(decimals=3)['surveyquestion1'][3] + df.round(decimals=3)['surveyquestion1'][2] + df.round(decimals=3)['surveyquestion1'][1])  

In [ ]:
dfnetscore1 = pd.DataFrame(netScore1).round(decimals=1)
dfnetscore1['dateyrmo'] = dfnetscore1.index
dfnetscore1.columns.values[0] = 'net_score_question_1'
dfnetscore1.reset_index(drop=True)[['net_score_question_1']]
d000 = pd.DataFrame(dfnetscore1['net_score_question_1'])
d000

In [ ]:
netScore2 = df.round(decimals=3)['surveyquestion2'][5] - (df.round(decimals=3)['surveyquestion2'][3] + df.round(decimals=3)['surveyquestion2'][2] + df.round(decimals=3)['surveyquestion2'][1])  

In [ ]:
dfnetscore2 = pd.DataFrame(netScore2).round(decimals=1)
dfnetscore2['dateyrmo'] = dfnetscore2.index
dfnetscore2.columns.values[0] = 'net_score_question_2'
dfnetscore2.reset_index(drop=True)[['net_score_question_2']]
d999 = pd.DataFrame(dfnetscore2['net_score_question_2'])
d999

In [ ]:
df